In [24]:
import pandas as pd
import numpy as np
import os
from sklearn.utils import shuffle
import sklearn 
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import random
import warnings
warnings.filterwarnings('ignore')




### Wheeze_188 Distribution

In [9]:
Total = np.unique(pd.read_csv("./188_wheeze.txt", header=None)[0].values)
name = []
for i in Total:
    name.append(i[:4])

unique = []
before_after = []
for i in Total:
    if np.sum((np.array(name, dtype = int) == int(i[:4]))) == 1:
        unique.append(i)
    else:
        before_after.append(i) 

Asthmatic_Female = []
Asthmatic_Male = []
Healthy_Male = []
Healthy_Female = []
for file in unique:
    if file.find('sthma') !=-1:
        if file.find("_M_")!=-1:
            Asthmatic_Male.append(file)
        if file.find("_F_")!=-1:
            Asthmatic_Female.append(file)
    if file.find("_C_")!=-1:
        if file.find("_M_")!=-1:
            Healthy_Male.append(file)
        if file.find("_F_")!=-1:
            Healthy_Female.append(file)

In [14]:
print(f"Total = {len(Total)}")
print(f"Asthmatic_male = {len(Asthmatic_Male)}")
print(f"Asthmatic_female = {len(Asthmatic_Female)}")
print(f"Healthy_Male = {len(Healthy_Male)}")
print(f"Healthy_female = {len(Healthy_Female)}")
print(f"Before_after = {len(before_after)}")

Total = 188
Asthmatic_male = 60
Asthmatic_female = 46
Healthy_Male = 42
Healthy_female = 34
Before_after = 6


### Breathe

In [16]:
Total = np.unique(os.listdir("./breathe_mfcc/"))
name = []
for i in Total:
    name.append(i[:4])

unique = []
before_after = []
for i in Total:
    if np.sum((np.array(name, dtype = int) == int(i[:4]))) == 1:
        unique.append(i)
    else:
        before_after.append(i) 

Asthmatic_Female = []
Asthmatic_Male = []
Healthy_Male = []
Healthy_Female = []
for file in unique:
    if file.find('sthma') !=-1:
        if file.find("_M_")!=-1:
            Asthmatic_Male.append(file)
        if file.find("_F_")!=-1:
            Asthmatic_Female.append(file)
    if file.find("_C_")!=-1:
        if file.find("_M_")!=-1:
            Healthy_Male.append(file)
        if file.find("_F_")!=-1:
            Healthy_Female.append(file)

In [17]:
print(f"Total = {len(Total)}")
print(f"Asthmatic_male = {len(Asthmatic_Male)}")
print(f"Asthmatic_female = {len(Asthmatic_Female)}")
print(f"Healthy_Male = {len(Healthy_Male)}")
print(f"Healthy_female = {len(Healthy_Female)}")
print(f"Before_after = {len(before_after)}")

Total = 258
Asthmatic_male = 78
Asthmatic_female = 55
Healthy_Male = 55
Healthy_female = 50
Before_after = 20


#### Helper functions

In [18]:
def train_test_set(Asthmatic_Female, Asthmatic_Male, Healthy_Female, Healthy_Male, before_after):
    random.seed(0)
    test_set = np.concatenate(((np.random.choice(np.array(Asthmatic_Male), replace=False, size=(7)),
                                np.random.choice(np.array(Asthmatic_Female), replace=False, size=(7)),
                                np.random.choice(np.array(Healthy_Male), replace=False, size=(7)),
                                np.random.choice(np.array(Healthy_Female), replace=False, size=(7)))))
    train_set = np.concatenate((np.array(Asthmatic_Male),
                                np.array(Asthmatic_Female),
                                np.array(Healthy_Male),
                                np.array(Healthy_Female)))
    train_set = np.setdiff1d(train_set,test_set)
#     np.array(before_after).shape
    train_set = np.concatenate((train_set, np.array(before_after)))
    return train_set, test_set

In [19]:
# train_test Csv
def train_test_csv(sound_dataframe):
    Total = np.unique(sound_dataframe.iloc[:,sound_dataframe.shape[1]-1])
    name = []
    for i in Total:
        name.append(i[:4])

    unique = []
    before_after = []
    for i in Total:
        if np.sum((np.array(name, dtype = int) == int(i[:4]))) == 1:
            unique.append(i)
        else:
            before_after.append(i) 

    Asthmatic_Female = []
    Asthmatic_Male = []
    Healthy_Male = []
    Healthy_Female = []
    for file in unique:
        if file.find('sthma') !=-1:
            if file.find("_M_")!=-1:
                Asthmatic_Male.append(file)
            if file.find("_F_")!=-1:
                Asthmatic_Female.append(file)
        if file.find("_C_")!=-1:
            if file.find("_M_")!=-1:
                Healthy_Male.append(file)
            if file.find("_F_")!=-1:
                Healthy_Female.append(file)

                

    Train, Test = train_test_set(Asthmatic_Female, Asthmatic_Male, Healthy_Female, Healthy_Male, before_after)     

    test_dataframe = pd.DataFrame()
    for i, line in enumerate(Test):
        A = sound_dataframe[(sound_dataframe[str(sound_dataframe.shape[1]-1)] == (Test[i]))]
        test_dataframe = pd.DataFrame.append(test_dataframe,A)
        
        
    train_dataframe = pd.DataFrame()
    for i, line in enumerate(Train):
        A = sound_dataframe[(sound_dataframe[str(sound_dataframe.shape[1]-1)] == (Train[i]))]
        train_dataframe = pd.DataFrame.append(train_dataframe,A) 
        
        
    
    return train_dataframe, test_dataframe

In [20]:
#Machine Learning Algorithm (MLA) Selection and Initialization
def MLA_selection(sound_dataframe, sound, path):    
    MLA = [
        #Ensemble Methods
#         ensemble.AdaBoostClassifier(),
#         ensemble.BaggingClassifier(),
#         ensemble.ExtraTreesClassifier(),
        ensemble.GradientBoostingClassifier(),
        ensemble.RandomForestClassifier(),

        #Gaussian Processes
#         gaussian_process.GaussianProcessClassifier(),

        #GLM
#         linear_model.LogisticRegressionCV(),
#         linear_model.PassiveAggressiveClassifier(),
#         linear_model.RidgeClassifierCV(),
#         linear_model.SGDClassifier(),
#         linear_model.Perceptron(),

        #Navies Bayes
#         naive_bayes.BernoulliNB(),
#         naive_bayes.GaussianNB(),

        #Nearest Neighbor
#         neighbors.KNeighborsClassifier(),

        #SVM
        svm.SVC(probability=True),
#         svm.NuSVC(probability=True),
        svm.LinearSVC(),

        #Trees    
#         tree.DecisionTreeClassifier(),
# #         tree.ExtraTreeClassifier(),

        #Discriminant Analysis
#         discriminant_analysis.LinearDiscriminantAnalysis(),
#         discriminant_analysis.QuadraticDiscriminantAnalysis(),


        XGBClassifier(max_depth = 8,
                     subsample = 0.8,
                     learning_rate = 0.01,
                     n_estimators = 450,
                     min_child_weight = 1)
#         XGBClassifier()
        ]


    MLA_columns = ['MLA Name', 'MLA Parameters','MLA Train Accuracy','MLA Train Accuracy Mean', 'MLA Test Accuracy', 'MLA Test Accuracy Mean','MLA Test Accuracy Std' ]
    MLA_compare = pd.DataFrame(columns = MLA_columns)
    row_index = 0
    for alg in MLA:
        MLA_name = alg.__class__.__name__
        MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
        MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())
        test = []
        train = []
        for i in tqdm(range(0,10)):
            train_csv, test_csv = pd.read_csv(path+'/train'+sound+str(i)+'.csv'), pd.read_csv(path+'./test'+sound+str(i)+'.csv')
            X_train = train_csv.iloc[:,:sound_dataframe.shape[1]-2]
            y_train = train_csv.iloc[:,sound_dataframe.shape[1]-2]
            X_test = test_csv.iloc[:,:sound_dataframe.shape[1]-2]
            y_test = test_csv.iloc[:,sound_dataframe.shape[1]-2]
            #Scaling
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)
            X_train = pd.DataFrame(X_train)
            X_test = pd.DataFrame(X_test)

            alg.fit(X_train,y_train)
            test.append(metrics.accuracy_score(y_test,alg.predict(X_test)))

            train.append(metrics.accuracy_score(y_train,alg.predict(X_train)))
            
#             if alg.__class__.__name__ == 'XGBClassifier':
#                 filename = './CMN/XGB_model'+ str(i) + '.model'
#                 pickle.dump(alg, open(filename, 'wb'))
#                 train_csv.to_csv('./CMN/Train_XGB'+str(i)+'.csv', index=False)
                
        
        MLA_compare.loc[row_index, 'MLA Train Accuracy'] = train
        MLA_compare.loc[row_index, 'MLA Train Accuracy Mean'] = np.mean(train)
        MLA_compare.loc[row_index, 'MLA Test Accuracy'] = test
        MLA_compare.loc[row_index, 'MLA Test Accuracy Mean'] = np.mean(test)
        MLA_compare.loc[row_index, 'MLA Test Accuracy Std'] = np.std(test)


        row_index+=1

    
    MLA_compare.sort_values(by = ['MLA Test Accuracy Mean'], ascending = False, inplace = True)
    
    return MLA_compare

In [41]:
# Making Sets
def make_and_save_sets(sound_dataframe, sound):
    for i in range(0,10):
        train_csv, test_csv = train_test_csv(sound_dataframe)
        train_csv.to_csv('./breathe_set_csv/train'+sound+str(i)+'.csv', index=False)
        test_csv.to_csv('./breathe_set_csv/test'+sound+str(i)+'.csv', index=False)

In [36]:
Wheeze_dataframe = pd.read_csv("./MFCCs_stasts_Csv/Wheeze.csv")
Breath_dataframe = pd.read_csv("./breath_csv/Breath.csv")


In [33]:
make_and_save_sets(Wheeze_dataframe, 'Wheeze')

In [34]:
MLA_Wheeze = MLA_selection(Wheeze_dataframe, 'Wheeze', "./Set_CSV/")

  0%|          | 0/10 [00:00<?, ?it/s]

[10:27:04] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 10%|█         | 1/10 [00:00<00:04,  1.83it/s]

[10:27:05] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 20%|██        | 2/10 [00:01<00:04,  1.99it/s]

...

 80%|████████  | 8/10 [00:03<00:00,  2.06it/s]

[10:27:08] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 90%|█████████ | 9/10 [00:04<00:00,  1.89it/s]

[10:27:09] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████| 10/10 [00:05<00:00,  1.96it/s]


In [35]:
MLA_Wheeze

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8202764976958525, 0.7991452991452992, 0.745...",0.845565,0.0604414
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8018433179723502, 0.8290598290598291, 0.75,...",0.838882,0.0488401
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9921203438395415, 0.9934735315445975, 0.997...",0.993245,"[0.8294930875576036, 0.7863247863247863, 0.754...",0.838231,0.0503668
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9770773638968482, 0.9746192893401016, 0.975...",0.97464,"[0.7926267281105991, 0.7435897435897436, 0.754...",0.801584,0.0579183
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.9075931232091691, 0.9071791153009428, 0.909...",0.903283,"[0.815668202764977, 0.7991452991452992, 0.7627...",0.7838,0.0577858


In [37]:
make_and_save_sets(Breath_dataframe, 'Breath')

In [38]:
MLA_Breath = MLA_selection(Breath_dataframe, 'Breath', "./Set_CSV/")

  0%|          | 0/10 [00:00<?, ?it/s]

[10:34:58] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 10%|█         | 1/10 [00:00<00:08,  1.12it/s]

[10:34:59] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 20%|██        | 2/10 [00:01<00:07,  1.14it/s]

...

 80%|████████  | 8/10 [00:07<00:01,  1.06it/s]

[10:35:06] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 90%|█████████ | 9/10 [00:08<00:00,  1.07it/s]

[10:35:07] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████| 10/10 [00:09<00:00,  1.09it/s]


In [40]:
Wheeze_dataframe = pd.read_csv("./breath_csv/Wheeze.csv")
Wheeze_dataframe_1 = pd.read_csv("./breath_csv/Wheeze_1.csv")
Wheeze_dataframe_2 = pd.read_csv("./breath_csv/Wheeze_2.csv")
Wheeze_dataframe_3 = pd.read_csv("./breath_csv/Wheeze_3.csv")

In [42]:
make_and_save_sets(Wheeze_dataframe, 'Wheeze')
make_and_save_sets(Wheeze_dataframe_1, 'Wheeze_1')
make_and_save_sets(Wheeze_dataframe_2, 'Wheeze_2')
make_and_save_sets(Wheeze_dataframe_3, 'Wheeze_3')

In [43]:
MLA_Wheeze = MLA_selection(Wheeze_dataframe, 'Wheeze', "./breathe_set_csv/")
MLA_Wheeze_1 = MLA_selection(Wheeze_dataframe_1, 'Wheeze_1', "./breathe_set_csv/")
MLA_Wheeze_2 = MLA_selection(Wheeze_dataframe_2, 'Wheeze_2', "./breathe_set_csv/")
MLA_Wheeze_3 = MLA_selection(Wheeze_dataframe_3, 'Wheeze_3', "./breathe_set_csv/")

  0%|          | 0/10 [00:00<?, ?it/s]

[14:13:14] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 10%|█         | 1/10 [00:00<00:05,  1.51it/s]

[14:13:14] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 20%|██        | 2/10 [00:00<00:03,  2.20it/s]

...

 80%|████████  | 8/10 [00:04<00:00,  2.28it/s]

[14:16:10] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 90%|█████████ | 9/10 [00:04<00:00,  2.40it/s]

[14:16:11] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████| 10/10 [00:05<00:00,  1.93it/s]


In [44]:
MLA_Wheeze

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8, 0.8433734939759037, 0.8847736625514403, ...",0.871286,0.0571511
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.796078431372549, 0.8353413654618473, 0.8847...",0.86717,0.0519343
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9955817378497791, 0.9926686217008798, 0.989...",0.992204,"[0.8156862745098039, 0.8393574297188755, 0.855...",0.858924,0.054189
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.9131075110456554, 0.9186217008797654, 0.902...",0.900739,"[0.7568627450980392, 0.7550200803212851, 0.831...",0.818151,0.0522078
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9808541973490427, 0.9794721407624634, 0.975...",0.974493,"[0.7490196078431373, 0.7951807228915663, 0.802...",0.812953,0.0461276


In [45]:
MLA_Wheeze_1

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7090163934426229, 0.8170731707317073, 0.799...",0.78963,0.0556047
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9715120525931337, 0.9678127286027798, 0.970...",0.97047,"[0.7131147540983607, 0.7804878048780488, 0.815...",0.78135,0.0457151
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7336065573770492, 0.7723577235772358, 0.811...",0.778935,0.0405394
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8714390065741418, 0.858814923189466, 0.8802...",0.869807,"[0.7377049180327869, 0.7682926829268293, 0.774...",0.765878,0.0281375
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9422936449963477, 0.9326993416239941, 0.936...",0.938048,"[0.7049180327868853, 0.7560975609756098, 0.807...",0.764156,0.0507447


In [46]:
MLA_Wheeze_2

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9898107714701602, 0.9884476534296028, 0.989...",0.990103,"[0.8326359832635983, 0.8771929824561403, 0.878...",0.831607,0.0352919
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8284518828451883, 0.8640350877192983, 0.907...",0.830254,0.0450774
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.970160116448326, 0.9638989169675091, 0.9665...",0.967557,"[0.8200836820083682, 0.868421052631579, 0.8529...",0.827346,0.0355962
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.803347280334728, 0.8640350877192983, 0.8571...",0.827132,0.0403313
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8966521106259098, 0.8859205776173286, 0.895...",0.896923,"[0.7615062761506276, 0.8421052631578947, 0.773...",0.79072,0.0442801


In [47]:
MLA_Wheeze_3

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9428571428571428, 0.9502196193265008, 0.948...",0.949912,"[0.7782258064516129, 0.7975708502024291, 0.757...",0.794569,0.0420262
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8747252747252747, 0.8777452415812591, 0.873...",0.880555,"[0.8387096774193549, 0.7975708502024291, 0.785...",0.792286,0.044346
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7903225806451613, 0.8178137651821862, 0.742...",0.774301,0.0570984
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9663003663003663, 0.9736456808199122, 0.967...",0.970274,"[0.7782258064516129, 0.8218623481781376, 0.771...",0.77331,0.0527904
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7782258064516129, 0.7813765182186235, 0.757...",0.768076,0.0536445


In [49]:
Wheeze_dataframe_0 = pd.read_csv("./breath_csv/Wheeze_0.csv")
Wheeze_dataframe_5 = pd.read_csv("./breath_csv/Wheeze_5.csv")
Wheeze_dataframe_10 = pd.read_csv("./breath_csv/Wheeze_10.csv")
Wheeze_dataframe_15 = pd.read_csv("./breath_csv/Wheeze_15.csv")
Wheeze_dataframe_20 = pd.read_csv("./breath_csv/Wheeze_20.csv")
Wheeze_dataframe_25 = pd.read_csv("./breath_csv/Wheeze_25.csv")
Wheeze_dataframe_30 = pd.read_csv("./breath_csv/Wheeze_30.csv")
Wheeze_dataframe_35 = pd.read_csv("./breath_csv/Wheeze_35.csv")
Wheeze_dataframe_40 = pd.read_csv("./breath_csv/Wheeze_40.csv")
Wheeze_dataframe_45 = pd.read_csv("./breath_csv/Wheeze_45.csv")
Wheeze_dataframe_50 = pd.read_csv("./breath_csv/Wheeze_50.csv")


In [50]:
make_and_save_sets(Wheeze_dataframe_0, 'Wheeze_0')
make_and_save_sets(Wheeze_dataframe_5, 'Wheeze_5')
make_and_save_sets(Wheeze_dataframe_10, 'Wheeze_10')
make_and_save_sets(Wheeze_dataframe_15, 'Wheeze_15')
make_and_save_sets(Wheeze_dataframe_20, 'Wheeze_20')
make_and_save_sets(Wheeze_dataframe_25, 'Wheeze_25')
make_and_save_sets(Wheeze_dataframe_30, 'Wheeze_30')
make_and_save_sets(Wheeze_dataframe_35, 'Wheeze_35')
make_and_save_sets(Wheeze_dataframe_40, 'Wheeze_40')
make_and_save_sets(Wheeze_dataframe_45, 'Wheeze_45')
make_and_save_sets(Wheeze_dataframe_50, 'Wheeze_50')

In [52]:
MLA_wheeze_0 = MLA_selection(Wheeze_dataframe_0, 'Wheeze_0', "./breathe_set_csv/")
MLA_wheeze_5 = MLA_selection(Wheeze_dataframe_5, 'Wheeze_5', "./breathe_set_csv/")
MLA_wheeze_10 = MLA_selection(Wheeze_dataframe_10, 'Wheeze_10', "./breathe_set_csv/")
MLA_wheeze_15 = MLA_selection(Wheeze_dataframe_15, 'Wheeze_15', "./breathe_set_csv/")
MLA_wheeze_20 = MLA_selection(Wheeze_dataframe_20, 'Wheeze_20', "./breathe_set_csv/")
MLA_wheeze_25 = MLA_selection(Wheeze_dataframe_25, 'Wheeze_25', "./breathe_set_csv/")
MLA_wheeze_30 = MLA_selection(Wheeze_dataframe_30, 'Wheeze_30', "./breathe_set_csv/")
MLA_wheeze_35 = MLA_selection(Wheeze_dataframe_35, 'Wheeze_35', "./breathe_set_csv/")
MLA_wheeze_40 = MLA_selection(Wheeze_dataframe_40, 'Wheeze_40', "./breathe_set_csv/")
MLA_wheeze_45 = MLA_selection(Wheeze_dataframe_45, 'Wheeze_45', "./breathe_set_csv/")
MLA_wheeze_50 = MLA_selection(Wheeze_dataframe_50, 'Wheeze_50', "./breathe_set_csv/")

  0%|          | 0/10 [00:00<?, ?it/s]

[16:17:06] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 10%|█         | 1/10 [00:02<00:24,  2.69s/it]

[16:17:08] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 20%|██        | 2/10 [00:03<00:12,  1.62s/it]

...

 80%|████████  | 8/10 [00:13<00:03,  1.72s/it]

[16:29:02] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 90%|█████████ | 9/10 [00:14<00:01,  1.75s/it]

[16:29:04] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████| 10/10 [00:16<00:00,  1.62s/it]


In [53]:
MLA_wheeze_0

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.987600291757841, 0.9885714285714285, 0.9866...",0.986183,"[0.8636363636363636, 0.812206572769953, 0.9122...",0.847258,0.0509225
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.871900826446281, 0.8262910798122066, 0.9389...",0.845907,0.0603973
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8842975206611571, 0.8169014084507042, 0.900...",0.841327,0.0476739
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8781911013858498, 0.8871428571428571, 0.871...",0.884601,"[0.8305785123966942, 0.7793427230046949, 0.854...",0.81235,0.0387027
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.950401167031364, 0.9571428571428572, 0.9444...",0.95194,"[0.8181818181818182, 0.8028169014084507, 0.877...",0.809764,0.0341584


In [63]:
MLA_wheeze_50

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9272030651340997, 0.8025751072961373, 0.9, ...",0.839842,0.0607281
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9763313609467456, 0.9869565217391304, 0.972...",0.9805,"[0.9080459770114943, 0.7982832618025751, 0.888...",0.834478,0.0583442
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.89272030651341, 0.7896995708154506, 0.88, 0...",0.823343,0.0637085
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8661242603550295, 0.8942028985507247, 0.870...",0.885361,"[0.9157088122605364, 0.7639484978540773, 0.872...",0.822803,0.0779035
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9615384615384616, 0.9702898550724638, 0.955...",0.965424,"[0.8812260536398467, 0.7682403433476395, 0.872...",0.814577,0.0713778
